In [1]:
import pandas as pd
import os
pd.set_option('display.max_rows', None)
#os.getcwd()
#print(open("Data/Level_1_Trial_1/resp_1000_Events_000.txt").read())

In [2]:
#Create empty dataframe with desired structure
processedDF = pd.DataFrame(columns=['Participant_ID','Level_No','Trial_No', 'Task_Duration', 'Mistake_Count', 'Baseline_HR', 'Baseline_GSR', 'Task_Avg_HR_Raw','Task_Avg_GSR_Raw', 'Task_Avg_RR_Raw'])

#iterate through participants, levels and trials
for i in range(1000,1006): #Participant number   
    
    #print('\nData for participant',i,'\n-----------------------------')      
    #Calculate baseline for 1000 participant
    baseline_hr_df = pd.read_csv('Data/Baseline/resp_' + str(i) + '_Events_001.txt',  sep='\t', lineterminator='\n')
    baseline_hr =  baseline_hr_df['Heart rate'].mean()
    #print('Baseline HR -', baseline_hr_df['Heart rate'].mean())
    baseline_gsr_df = pd.read_csv('Data/Baseline/resp_' + str(i) + '_Events_003.txt',  sep='\t', lineterminator='\n')
    baseline_gsr = baseline_gsr_df['GSR Conductance CAL'].mean()
    #print('Baseline GSR -', baseline_gsr_df['GSR Conductance CAL'].mean())
      
    for j in range(1, 7): #Level number
        for k in range(1, 4): #Trial number
            loc_str = 'Data/Level_' + str(j) + '_Trial_' + str(k)
            eventMarkerdf = pd.read_csv((loc_str + '/resp_' + str(i) + '_Events_004.txt'),  sep='\t', lineterminator='\n')
            hr_df = pd.read_csv((loc_str + '/resp_' + str(i) + '_Events_001.txt'),  sep='\t', lineterminator='\n')
            gsr_df = pd.read_csv((loc_str + '/resp_' + str(i) + '_Events_003.txt'),  sep='\t', lineterminator='\n')
            #print(eventMarkerdf)
            #df.at[0,'Id:L;Name:StartTask;Key:A']
            task_start_time = eventMarkerdf.loc[eventMarkerdf['Data'] == 'Id:L;Name:StartTask;Key:A']['TimeStamp'].values[0]
            task_end_time = eventMarkerdf.loc[eventMarkerdf['Data'] == 'Id:L;Name:EndTask;Key:D']['TimeStamp'].values[0]
            
            mistakeAlldf = pd.read_csv((loc_str + '/resp_' + str(i) + '_Events_002.txt'),  sep='\t', lineterminator='\n')
            #print(mistakeAlldf.shape)
            mistakeTimeStampsdf = mistakeAlldf[mistakeAlldf['TimeStamp'].between(task_start_time, task_end_time)]['TimeStamp']
            mistakeCount = 0
            #print(mistakeTimeStampsdf.shape)   
            for m in range(1,len(mistakeTimeStampsdf)): 
                #print('Currently processing ', mistakeTimeStampsdf.iloc[i])
                timeDiff = mistakeTimeStampsdf.iloc[m] - mistakeTimeStampsdf.iloc[m-1]
                #print('Difference with last time stamp- ', timeDiff)
                if(timeDiff > 40): #mistake detected if within the timeDiff threshold
                    mistakeCount+=1
        
            mistakeCount += 1 #For counting the last mistake which is ignored by the above logic   
            
            task_time_secs = (task_end_time - task_start_time) / 1000 
            
            avg_raw_hr = hr_df[hr_df['TimeStamp'].between(task_start_time, task_end_time)]['Heart rate'].mean()
            avg_raw_rr_interval = hr_df[hr_df['TimeStamp'].between(task_start_time, task_end_time)]['R-R interval'].mean()
            avg_raw_gsr = gsr_df[gsr_df['TimeStamp'].between(task_start_time, task_end_time)]['GSR Conductance CAL'].mean() 
            #print('Data for Level',j,', Trial',k)
            #print('Time taken for trial - ', task_time_secs)
            #print('Average HR (Raw) - ', avg_raw_hr)
            #print('Average GSR (Raw) - ', avg_raw_gsr)
            processedDF = processedDF.append({'Participant_ID': i, 'Level_No': j, 'Trial_No': k, 'Task_Duration': task_time_secs, 'Mistake_Count': mistakeCount, 'Baseline_HR': baseline_hr, 'Baseline_GSR': baseline_gsr, 'Task_Avg_HR_Raw': avg_raw_hr, 'Task_Avg_GSR_Raw': avg_raw_gsr, 'Task_Avg_RR_Raw' : avg_raw_rr_interval}, ignore_index=True)
            #df.iloc[0]['Id:L;Name:StartTask;Key:A']
            
#print(processedDF)
processedDF.to_csv('processedDF.csv', sep='\t')